In [3]:
import numpy as np
import pandas as pd
import networkx as nx


In [1]:
# Check that the main package has been installed (with pip install -e .)
import src


In [4]:
# Notebook configuration and utility functions:
from IPython.display import display

pd.options.display.max_rows = 100

In [5]:
# Download data: (Requires kaggle login)

# Option 1: Download manually from https://www.kaggle.com/carrie1/ecommerce-data
# * Unzip and place csv file in data/raw/data.csv

# Option 2: Download automatically using Kaggle API 

# Options for storing credentials: environment variables or user-config file.
# Auth in environment vars:
#   export KAGGLE_USERNAME=datadinosaur
#   export KAGGLE_KEY=xxxxxxxxxxxxxx
# Auth in kaggle.json file: 
# Download kaggle.json from account page on kaggle.com and place in ~/.kaggle/kaggle.json
# pip install kaggle

# import kaggle
# kaggle.api.authenticate()
# kaggle.api.dataset_download_files(
#     'The_name_of_the_dataset', 
#     path='data/raw', 
#     unzip=True
# )

In [21]:
# Load data
input_fn = "../data/raw/data.csv"
# df = df_raw = pd.read_csv("../data/raw/data.csv")
# keep_default_na=False Giver mere kontrol over tomme strings.
# encoding='Latin-1' (ISO-8859-1) is sometimes needed. If strings still look weird, try  ISO-8859-15.
# (Problematic byte is '£', e.g. Dotcomgiftshop Gift Voucher £40.00 in line 38250).
# Sometimes you can also just to ask Pandas to use python engine instead of C with `engine='python'`.
df = df_raw = pd.read_csv("../data/raw/data.csv", keep_default_na=False, encoding='Latin-1')   # Works
# df = df_raw = pd.read_csv("../data/raw/data.csv", keep_default_na=False, engine='python')
print(f"Table rows:", len(df))
# df.head()
df.loc[38245:38255]

Table rows: 541909


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
38245,539492,90176B,DIAMANTE NECKLACE BLACK,1,12/20/2010 10:14,7.64,,United Kingdom
38246,539492,90197B,BLACK GLASS BRACELET W HEART CHARMS,1,12/20/2010 10:14,5.09,,United Kingdom
38247,539492,90209C,PINK ENAMEL+GLASS HAIR COMB,1,12/20/2010 10:14,2.11,,United Kingdom
38248,539492,gift_0001_40,Dotcomgiftshop Gift Voucher £40.00,1,12/20/2010 10:14,34.04,,United Kingdom
38249,539492,DOT,DOTCOM POSTAGE,1,12/20/2010 10:14,396.81,,United Kingdom
38250,539492,21481,FAWN BLUE HOT WATER BOTTLE,11,12/20/2010 10:14,2.95,,United Kingdom
38251,539493,22585,PACK OF 6 BIRDY GIFT TAGS,2,12/20/2010 10:15,1.25,,United Kingdom
38252,539493,21111,"SWISS ROLL TOWEL, CHOCOLATE SPOTS",1,12/20/2010 10:15,1.48,,United Kingdom
38253,539493,21110,LARGE CAKE TOWEL PINK SPOTS,1,12/20/2010 10:15,3.38,,United Kingdom
38254,539493,72799F,IVORY PILLAR CANDLE GOLD FLOCK,1,12/20/2010 10:15,2.95,,United Kingdom


In [19]:
# If you get a 
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa3 in position 79780: invalid start byte
# The problematic byte is the British pound sign, '£'.
# Can be solved by selecting , encoding='Latin-1'
# OBS: Pandas reads the file in chunks, so is likely not position 79780 in the actual file.
input_fn = "../data/raw/data.csv"
err_pos = 79780
with open(input_fn, mode="rb") as f:
    f.seek(err_pos-20)
    s = f.read(40)
    print(s)
    print("  "  + "0123456789"*4)



b'TLE,2,12/1/2010 12:35,3.95,17377,United '
  0123456789012345678901234567890123456789


In [ ]:
# Remove bad data:
df = df_raw
print("Rows before QA filtering:", len(df))

# Remove lines with N/A values:
for column in df.columns:
    print(f" - NaN {column}:", df[column].isna().sum()) 
df = df_dropna = df.dropna()
print("Rows after dropping N/A:", len(df))

print("Columns with empty values:")
for column in df.columns:
    print(f" - {column} empty values:", (df[column] == '').sum()) 
    df = df[~(df[column] == '')]
print("Rows after dropping rows with empty values:", len(df))
    
# Remove lines that does not represent an actual product:
print("Non-product stock codes:")
non_product_stock_codes = ['BANK CHARGES', 'C2', 'CRUK', 'D', 'DOT', 'M', 'PADS', 'POST']
for code in non_product_stock_codes:
    print(f" - {code} lines:", (df['StockCode'] == code).sum()) 
df = df[~df['StockCode'].isin(['BANK CHARGES', 'C2', 'CRUK', 'D', 'DOT', 'M', 'PADS', 'POST'])]
print("Rows after dropping non-product lines:", len(df))

# # print(f" - Number of POSTAGE lines:", len(df['StockCode'] == "POST"))
# print(f" - No. of POSTAGE lines:", (df['StockCode'] == "POST").sum())
# df = df[df['StockCode'] != "POST"]
# print("Rows after removing POST:", len(df))

print("Rows after QA filtering:", len(df))

df.head()

In [ ]:
# Reduce table size for dev/testing:
# df_org = df
# df = df[:10000]

In [ ]:
# Per-order stats:

grouped_by_invoiceno = df.groupby("InvoiceNo")

# Number of different products per order (not including quantity)
order_sizes = grouped_by_invoiceno.size()
axes = grouped_by_invoiceno.size().hist(bins=order_sizes.max(), figsize=(12, 4))



In [ ]:
# Interestingly, we have a few orders containing a great number of different products.
axes.set_xlim(0, 100)
# axes.set_ylim(0, 20)
display(axes.figure)

In [ ]:
# Interestingly, we have a few orders containing a great number of different products.
axes.set_xlim(100, 600)
axes.set_ylim(0, 20)
display(axes.figure)

In [ ]:
# What is the largest order?

print("Number of items in order with most unique items:", grouped_by_invoiceno.size().max())

# invoice_with_most_unique_items = grouped_by_invoiceno.size().argmax()  # This is the row number
invoice_with_most_unique_items = grouped_by_invoiceno.size().idxmax()  # This is the index

print(invoice_with_most_unique_items)

# grouped_by_invoiceno.groups[invoice_with_most_unique_items]
with pd.option_context('display.max_rows', 100):
    # Need to explicitly use df.head(), otherwise Jupyter Notebook is in charge of selecting table size
    display(grouped_by_invoiceno.get_group(invoice_with_most_unique_items).head(20))



# This seems to be an adhoc invoice for some data with no customer ID?


In [ ]:
type(grouped_by_invoiceno)

In [ ]:
print("Number of rows with N/A CustomerID:", df['CustomerID'].isna().sum())
df[df['CustomerID'].isna()]

In [ ]:
print("Number of lines with N/A Description:", df['Description'].isna().sum())
df[df['Description'].isna()]['CustomerID'].isna().sum()

In [ ]:
print("Are all lines with N/A Description also with N/A CustomerID?")
df[df['Description'].isna()]['CustomerID'].isna().sum()
print("Yes.")

df[df['Description'].isna()]['CustomerID'].dropna()


In [ ]:
nx.Graph.to_net

In [ ]:
import timeit
timeit.timeit?
# timeit.Timer.timeit?

In [ ]:
# How does the product codes look?
# "POST" is the most frequent?
# What is that?

with pd.option_context('display.max_rows', 100):
    display(df['StockCode'].sort_values().tail(100))

In [ ]:
# POSTAGE is applied to countries outside of UK:

print(f" - Number of POSTAGE lines:", (df['StockCode'] == "POST").sum())

df_post = df[df['StockCode'] == "POST"]
display(df_post)

df_post['Country'].unique()

In [ ]:
# Lad os se hvad folk i Danmark har købt:
df[df['Country'] == "Denmark"]

In [ ]:
# Other weird StockCodes:
# 'BANK CHARGES', 'C2', 'CRUK', 'D', 'DOT', 'M', 'PADS'

df[df['StockCode'] == 'C2']

In [ ]:
df[df['StockCode'] == 'CRUK']


In [ ]:

df_weird_stockcodes = df[df['StockCode'].isin(['BANK CHARGES', 'C2', 'CRUK', 'D', 'DOT', 'M', 'PADS', 'POST'])]

print("Rows with weird stock codes:", len(df_weird_stockcodes))
df_weird_stockcodes

In [ ]:
print(len(df))

len(df[~df['StockCode'].isin(['BANK CHARGES', 'C2', 'CRUK', 'D', 'DOT', 'M', 'PADS', 'POST'])])

In [ ]:
non_product_stock_codes = ['BANK CHARGES', 'C2', 'CRUK', 'D', 'DOT', 'M', 'PADS', 'POST']

for stockcode in non_product_stock_codes:
    display(df[df['StockCode'] == stockcode])


In [ ]:
df['StockCode'].unique()

In [ ]:
all_stockcodes = np.sort(df['StockCode'].unique())
print(", ".join(all_stockcodes[-100:]))


In [ ]:
df.sort_values('StockCode').tail(20)
df_pr

In [ ]:
products_by_orders_count = df.groupby('StockCode')['InvoiceNo'].count().sort_values(ascending=False)
products_by_orders_count

In [ ]:
# Først, lav en tabel med alle produkter.
sales_df = df
products_df = sales_df.groupby('StockCode').first()
del products_df['InvoiceNo']
del products_df['Quantity']
del products_df['CustomerID']
del products_df['Country']

# Beregn product order count:
# product_orders_count = df.groupby('StockCode')['InvoiceNo'].count().sort_values(ascending=False)
products_df['OrdersCount'] = sales_df.groupby('StockCode')['InvoiceNo'].count()

products_df_sorted = products_df.sort_values(by='OrdersCount', ascending=False)

products_df_sorted.tail(20)

In [ ]:
sales_df.groupby('StockCode').first().sort_values(by='StockCode').tail(20)

In [ ]:
products_df_sorted.index.array
dir(products_df_sorted.index)

In [ ]:
type(products_df_sorted.index[0])

In [ ]:
products_df_sorted.at[]

In [ ]:
[att for att in dir(pd.Series) if 'sort' in att]

In [ ]:
df.groupby("InvoiceNo").size().value_counts().sort_index().head(20)


In [ ]:
df.groupby("InvoiceNo").size().value_counts().sort_index()[0:100].plot.line()

In [ ]:
len(df.groupby("InvoiceNo"))

In [ ]:
# Build same-order undirected graph:
# dict[StockCode1][StockCode2] = count

grouped_by_invoiceno = df.groupby("InvoiceNo")

# print("Total orders (invoices):", grouped_by_invoiceno.ngroups)
print("Dataset:")
print(f"- {len(df)} rows,")
print(f"- {df.groupby('InvoiceNo').ngroups} orders/invoices,")
print(f"- {len(df['StockCode'].unique())} unique StockCodes.")

def build_sameorder_product_dod_using_cartesian_forloop(df):
    # We use a double default-dict, but since most StockCodes are present, 
    # it is probably faster to pre-populate the outer dict with *all* StockCodes.
    # Also, we only have 3684, so it would be possible to use an adjacency matrix, 
    # instead of adjacency list - only about 10M values so about 40-80 MB. 
    
    sameorder_dod = defaultdict(lambda: defaultdict(int))
    for invoiceno, order_df in df.groupby("InvoiceNo"):
        # Can use either itertools.product or itertools.combinations:
        # itertools.combinations only gives unique combinations, but can do more than two.
        for stock_code1, stock_code2 in itertools.product(order_df['StockCode'], repeat=2):
            if stock_code1 == stock_code2:
                continue
            sameorder_dod[stock_code1][stock_code2] += 1
    return sameorder_dod


# Let's measure performance of using 
# for-loop over product(order_df['StockCode'], order_df['StockCode']), for each order, grouped by InvoiceNo:
# Time to compute:
ttc = timeit.timeit(
    'global sameorder_g_dod; '
    'sameorder_g_dod = build_sameorder_product_dod_using_cartesian_forloop(df)', 
    globals=locals(), 
    number=1
)

print(f"\nTime to compute same-order dict-of-dict, "
      f"using for-loop on cartesian product of StockCodes in each order: {ttc:.01f} sec")
print(" - single-threaded and inefficient for-loop.")
print(" - number of nodes:", len(sameorder_g_dod), "(single-product orders do not contribute to same-order graph)")
# sameorder_g_dod = build_sameorder_product_dod(df)


In [ ]:
weights = [1, 3, 8, 4, 5, 0.5]
weights = np.array(weights)
sidxs = np.argsort(weights)
weights[np.argsort(weights)]

In [ ]:
# What were the rows with N/A CustomerID

df_raw[df_raw['CustomerID'].isna()]

# row 622 / line 624: Empty string.
# 536414,22139,,56,12/1/2010 11:52,0,,United Kingdom


In [ ]:
timeit.default_timer()

In [ ]:

df[(df['CustomerID'] == '12583')]
df[(df['CustomerID'] == '12583') & (df['StockCode'] == '142')]

# Index is original df row index

df.loc[31]
df.loc[142]

In [ ]:
with pd.option_context('display.max_rows', 100):
    display(df[df['StockCode'] == '10002'].head(50))

In [ ]:
# with pd.option_context('display.max_rows', 100):
#     next(iter(df.groupby('StockCode')['CustomerID']))[1].head(50)

code, gs = next(iter(df.groupby('StockCode')['CustomerID']))
print(f"CustomerIDs for StockCode {code!r}:")
print(f"{gs.unique()=}")
print(f"{len(gs)=}")
print(f"{len(gs.unique())=}")
print(f"{gs.nunique()=}")
print(f"{gs.size=}")

display(gs.head(50))

gs.value_counts()

In [ ]:
pandas.core.groupby.generic.SeriesGroupBy.unique?

In [ ]:
# Beregn product order count:
# product_orders_count = df.groupby('StockCode')['InvoiceNo'].count().sort_values(ascending=False)
product_orders_count = df.groupby('StockCode')['InvoiceNo'].count()
products_df['OrdersCount'] = product_orders_count

# Beregn hvor mange brugere har købt hvert enkelt produkt:
product_users_count = df.groupby(by='StockCode')['CustomerID'].nunique()
# product_users_count = df.groupby(by='StockCode')['CustomerID'].value_counts()
display(product_users_count)

products_df['UsersCount'] = product_users_count


# for code, gdf in df.groupby('StockCode')['CustomerID']:
#     print(len(gdf))
#     print(len(gdf.unique()))
#     print(gdf.nunique())
#     display(gdf)
#     break
display(products_df)

In [ ]:
# Forskel: Antal brugere vs antal orders:
products_df['OrdersCount'] - products_df['UsersCount']
(products_df['OrdersCount'] - products_df['UsersCount']).max()
# Uh, det lader til at ingen kunder har købt den samme ting to gange??


In [ ]:
# Alternativ beregning, tjek om der er nogen brugere som har købt det samme produkt flere gange:
users_product_count = df.groupby('CustomerID')['StockCode'].count().sort_values(ascending=False)
users_product_count.head(20)

In [ ]:
df[df['CustomerID'] == '17841'].sort_values('StockCode')

In [ ]:
df[df['CustomerID'] == '17841'].sort_values('StockCode')

In [ ]:
df[df['StockCode'] == '15044A'].sort_values('CustomerID')

In [ ]:
df_15044A_sby_customer = df[df['StockCode'] == '15044A'].sort_values('CustomerID')
print("Antal salg af StockCode '15044A':", len(df_15044A_sby_customer))
print("Antal unikke kunder for '15044A':", df_15044A_sby_customer['CustomerID'].nunique())

display(df_15044A_sby_customer.head())

display(df_15044A_sby_customer.groupby('CustomerID').count())
display(df_15044A_sby_customer.groupby('CustomerID')['InvoiceNo'].count())
display(df_15044A_sby_customer.groupby('CustomerID').size())


# Konklusion: Der er helt sikker kunder der har købt det samme produkt flere gange.

In [22]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [25]:
df_gby_user = df.groupby('CustomerID')
# df_gby_user['17850']
df_gby_user.get_group('17850')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom
...,...,...,...,...,...,...,...,...
86898,C543611,82483,WOOD 2 DRAWER CABINET WHITE FINISH,-1,2/10/2011 14:38,4.95,17850,United Kingdom
86899,C543611,21874,GIN AND TONIC MUG,-1,2/10/2011 14:38,1.06,17850,United Kingdom
86900,C543611,71477,COLOUR GLASS. STAR T-LIGHT HOLDER,-2,2/10/2011 14:38,2.75,17850,United Kingdom
86901,C543611,82483,WOOD 2 DRAWER CABINET WHITE FINISH,-1,2/10/2011 14:38,4.95,17850,United Kingdom


In [26]:
pd.__version__

'1.3.0'